In [8]:
physOnto = [
    # Experiment
        {
            "type": "class",
            "name": "ExperimentSetup",
        },

        {
            "type": "class",
            "name": "ExperimentalDevice",
        },
        {
            "type": "property",
            "name": "ExperimentalDeviceHasSetup",
            "domain": "ExperimentalDevice",
            "range": "ExperimentSetup",
        },

        {
            "type": "class",
            "name": "ConductedExperiment",
        },
        {
            "type": "property",
            "name": "experimentConductedWithDevice",
            "domain": "ConductedExperiment",
            "range": "ExperimentalDevice",
        },
        {
            "type": "class",
            "name": "DateTime"
        },
        {
            "type": "property",
            "name": "experimentConductedAtTime",
            "domain": "ConductedExperiment",
            "range": "DateTime",
        },
    # Data
        {
            "type": "class",
            "name": "DataFormat",
        },
        {
            "type": "property",
            "name": "experimentHasDataFormat",
            "domain": "ConductedExperiment",
            "range": "DataFormat",
        },
        {
            "type": "property",
            "name": "experimentOutputHasDataFormat",                
            "domain": "ExperimentSetup",
            "range": "DataFormat",
        },
        {
            "type": "property",
            "name": "experimentInputHasDataFormat",                
            "domain": "ExperimentSetup",
            "range": "DataFormat",
        },

        {
            "type": "class",
            "name": "Data", 
        },
        {
            "type": "property",
            "name": "dataHasFormat",
            "domain": "Data",
            "range": "DataFormat",
        },
        {
            "type": "property",
            "name": "experimentOutputHasData",                
            "domain": "ConductedExperiment",
            "range": "Data",
        },
        {
            "type": "property",
            "name": "experimentInputHasData",                
            "domain": "ConductedExperiment",
            "range": "Data",
        }
    # Model
        {
            "type": "class",
            "name": "PhysicalModel",
        },
        {
            "type": "class",
            "name": "SimulatedExperiment",
        }
]

In [11]:
from owlready2 import *
def convertOntoJSONIntoOwlready2(ontojson, iri):
    onto = get_ontology(iri)
    with onto:
        for obj in ontojson:
            if obj["type"] == "class":
                NewClass = types.new_class(obj["name"], (Thing,))
            elif obj["type"] == "property":
                NewProperty = types.new_class(obj["name"], (ObjectProperty,))
                NewProperty.domain = [getattr(onto, obj["domain"])]
                NewProperty.range = [getattr(onto, obj["range"])]
    return onto

In [10]:
onto = convertOntoJSONIntoOwlready2(physOnto, "http://quantsimulant.de/physOnto.owl")

NameError: name 'physOnto' is not defined

In [12]:
from pathlib import Path
datapath = Path("../master-database-files/master-experimental/physics_ontology/")
datapath.mkdir(exist_ok=True, parents=True)

In [13]:
onto.save(file = str(datapath / "physOnto.owl"), format = "rdfxml")

NameError: name 'onto' is not defined

In [14]:
physci  = get_ontology("file://" + str(datapath / "physci.nt")).load()

* Owlready2 * WARNING: ObjectProperty http://purl.org/dc/terms/contributor belongs to more than one entity types: [owl.AnnotationProperty, owl.ObjectProperty, 1.1.contributor]; I'm trying to fix it...
* Owlready2 * WARNING: ObjectProperty http://purl.org/dc/terms/creator belongs to more than one entity types: [owl.AnnotationProperty, owl.ObjectProperty, 1.1.creator, terms.contributor]; I'm trying to fix it...
* Owlready2 * WARNING: ObjectProperty http://purl.org/dc/terms/medium belongs to more than one entity types: [owl.AnnotationProperty, owl.ObjectProperty, 1.1.format, terms.format]; I'm trying to fix it...
* Owlready2 * WARNING: ObjectProperty http://purl.org/dc/terms/publisher belongs to more than one entity types: [owl.AnnotationProperty, owl.ObjectProperty, 1.1.publisher]; I'm trying to fix it...
* Owlready2 * WARNING: ObjectProperty http://purl.org/dc/terms/type belongs to more than one entity types: [owl.AnnotationProperty, owl.ObjectProperty, 1.1.type]; I'm trying to fix it..

In [13]:
cls = [*physci.classes()]

In [27]:
def returnClassInformation(cls):
    clsInfo = {}
    clsInfo["name"] = cls.name
    clsInfo["iri"] = cls.iri
    clsInfo["description"] = cls.comment
    clsInfo["comment"] = cls.comment.en
    return clsInfo

In [28]:
returnClassInformation(cls[36])

{'name': 'Theorem',
 'iri': 'https://w3id.org/skgo/physci#Theorem',
 'comment': ['The term is used especially in mathematics where the axioms are those of mathematical logic and the systems in question.']}

In [15]:
from rdflib import Graph, URIRef, Literal, Namespace, term

In [16]:
physciGraph = Graph()
physciGraph.parse(str(datapath / "physci.nt"))
#purl = Namespace("http://purl.org/dc/elements/1.1/")
#physciGraph.bind("dc", purl)
# Search for all owl classes
qres = physciGraph.query(
    """SELECT ?s
       WHERE {
          ?s a owl:Class.
       }""")
physciClasses = []
for row in qres:
    # Continue if the class is a blank node
    if type(row.s) == term.BNode:
        continue
    newClass = {}
    physciClasses.append(newClass)
    newClass["iri"] = str(row.s)
    # Get the name from the iri
    newClass["name"] = row.s.split("#")[-1]
    # Search for the comment
    qresComment = physciGraph.query(
        """SELECT ?comment
           WHERE {
              ?s rdfs:comment ?comment.
           }""", initBindings = {'s': row.s})
    for rowComment in qresComment:
        # Get the english string of the comment
        newClass["comment"] = rowComment.comment.value
    # Search for the description with the predicate http://purl.org/dc/elements/1.1/description
    qresDescription = physciGraph.query(
        """SELECT ?description
           WHERE {
              ?s <http://purl.org/dc/elements/1.1/description> ?description.
           }""", initBindings = {'s': row.s})
    for rowDescription in qresDescription:
        # Get the english string of the description
        newClass["description"] = rowDescription.description.value
        print(newClass["description"])
physciProperties = []
# Search for all owl object properties
qres = physciGraph.query(
    """SELECT ?s
       WHERE {
          ?s a owl:ObjectProperty.
       }""")
for row in qres:
    # Continue if the class is a blank node
    if type(row.s) == term.BNode:
        continue
    newProperty = {}
    physciProperties.append(newProperty)
    newProperty["iri"] = str(row.s)
    # Get the name from the iri
    newProperty["name"] = row.s.split("#")[-1]
    # Search for the domain
    qresDomain = physciGraph.query(
        """SELECT ?domain
           WHERE {
              ?s rdfs:domain ?domain.
           }""", initBindings = {'s': row.s})
    for rowDomain in qresDomain:
        # Get the english string of the domain
        newProperty["domain"] = rowDomain.domain
    # Search for the range
    qresRange = physciGraph.query(
        """SELECT ?range
           WHERE {
              ?s rdfs:range ?range.
           }""", initBindings = {'s': row.s})
    for rowRange in qresRange:
        # Get the english string of the range
        newProperty["range"] = rowRange.range
    # Search for the comment
    qresComment = physciGraph.query(
        """SELECT ?comment
           WHERE {
              ?s rdfs:comment ?comment.
           }""", initBindings = {'s': row.s})
    for rowComment in qresComment:
        # Get the english string of the comment
        newProperty["comment"] = rowComment.comment.value
    # Search for the description with the predicate http://purl.org/dc/elements/1.1/description
    qresDescription = physciGraph.query(
        """SELECT ?description
           WHERE {
              ?s <http://purl.org/dc/elements/1.1/description> ?description.
           }""", initBindings = {'s': row.s})
    for rowDescription in qresDescription:
        # Get the english string of the description
        newProperty["description"] = rowDescription.description.value
        print(newProperty["description"])
print(physciClasses)
print(physciProperties)

Failed to convert Literal lexical form to value. Datatype=http://www.w3.org/2001/XMLSchema#date, Converter=<function parse_date at 0x7f96866b4d60>
Traceback (most recent call last):
  File "/home/gratach/Dokumente/vscode/venv/lib/python3.11/site-packages/rdflib/term.py", line 2119, in _castLexicalToPython
    return conv_func(lexical)  # type: ignore[arg-type]
           ^^^^^^^^^^^^^^^^^^
  File "/home/gratach/Dokumente/vscode/venv/lib/python3.11/site-packages/isodate/isodates.py", line 203, in parse_date
    raise ISO8601Error('Unrecognised ISO 8601 date format: %r' % datestring)
isodate.isoerror.ISO8601Error: Unrecognised ISO 8601 date format: '2.0'
Failed to convert Literal lexical form to value. Datatype=http://www.w3.org/2001/XMLSchema#date, Converter=<function parse_date at 0x7f96866b4d60>
Traceback (most recent call last):
  File "/home/gratach/Dokumente/vscode/venv/lib/python3.11/site-packages/rdflib/term.py", line 2119, in _castLexicalToPython
    return conv_func(lexical)  #

Something that is unstated but considered to be fact for the purpose of making a decision or solving a problem in a research.
An instance of a particular situation of researchwork; an example of something occurring.
In physics, defining equations are equations that define new quantities in terms of base quantities
Studies that are concerned with the observation of physical phenomena and experiments.
Mathematical expression is a finite combination of symbols that is well-formed according to rules that depend on the context.
A function was originally the idealization of how a varying quantity depends on another quantity.
a system or standard of measurement.
Observational data is a valuable form of research that can give researchers information that goes beyond numbers and statistics.
A room or building housing an astronomical telescope or other scientific equipment for the study of natural phenomena.
The term observer  used as detector which detect formations also as a special relativity

In [58]:
physciClasses[10]["comment"].value

'Radiance and spectral radiance are radiometric measures that describe the amount of light that passes through or is emitted from a particular area, and falls within a given solid angle in a specified direction. They are used to characterize both emission from diffuse sources and reflection from diffuse surfaces. The SI unit of radiance is watts per steradian per square metre (W·sr−1·m−2).'

In [97]:
def getAllClasses(graph):
    qres = graph.query(
        """SELECT ?s
           WHERE {
              ?s a owl:Class
              }""")
    classes = []
    for row in qres:
        if type(row.s) == term.BNode:
            continue
        classes.append(str(row.s))
    return classes

In [18]:
def getNodeName(graph, iri):
    return iri.replace("/", "#").split("#")[-1]

In [92]:
def getNodeDescription(graph, iri):
    qres = graph.query(
        """SELECT ?description
           WHERE {
              ?s <http://purl.org/dc/elements/1.1/description> ?description.
           }""", initBindings = {'s': URIRef(iri)})
    for row in qres:
        return row.description.value

In [91]:
def getNodeComment(graph, iri):
    qres = graph.query(
        """SELECT ?comment
           WHERE {
              ?s rdfs:comment ?comment.
           }""", initBindings = {'s': URIRef(iri)})
    for row in qres:
        return row.comment.value

In [105]:
def getSuperClasses(graph, iri):
    qres = graph.query(
        """SELECT ?superclass
           WHERE {
              ?s rdfs:subClassOf ?superclass.
              }""", initBindings = {'s': URIRef(iri)})
    superclasses = set()
    for row in qres:
        superclasses.add(str(row.superclass))
    returnSuperclasses = set()
    for superclass in superclasses:
        returnSuperclasses.add(superclass)
        returnSuperclasses = returnSuperclasses.union(getSuperClasses(graph, superclass))
    return returnSuperclasses

In [103]:
def getPropertyTriplesWithSubject(graph, iri):
    superclasses = getSuperClasses(graph, iri)
    triples = []
    for superclass in superclasses:
        qres = graph.query(
            """SELECT ?predicate ?object
                WHERE {
                    ?predicate a owl:ObjectProperty.
                    ?predicate rdfs:domain ?subject.
                    ?predicate rdfs:range ?object.
                }""", initBindings = {'subject': URIRef(superclass)})
        for row in qres:
            triples.append((superclass, str(row.predicate), str(row.object)))
    return triples

In [109]:
def getPropertyTriplesWithObject(graph, iri):
    superclasses = getSuperClasses(graph, iri)
    triples = []
    for superclass in superclasses:
        qres = graph.query(
            """SELECT ?predicate ?subject
                WHERE {
                    ?predicate a owl:ObjectProperty.
                    ?predicate rdfs:domain ?subject.
                    ?predicate rdfs:range ?object.
                }""", initBindings = {'object': URIRef(superclass)})
        for row in qres:
            triples.append((str(row.subject), str(row.predicate), superclass))
    return triples

In [1]:
def getAllNamedIndividuals(graph):
    qres = graph.query(
        """SELECT ?s
           WHERE {
              ?s a owl:NamedIndividual
              }""")
    namedIndividuals = []
    for row in qres:
        if type(row.s) == term.BNode:
            continue
        namedIndividuals.append(str(row.s))
    return namedIndividuals

In [2]:
def getAllConnectionsBetweenNamedIndividuals(graph):
    qres = graph.query(
        """SELECT ?subject ?predicate ?object
           WHERE {
                ?subject ?predicate ?object.
                ?subject a owl:NamedIndividual.
                ?object a owl:NamedIndividual.
              }""")
    connections = []
    for row in qres:
        connections.append((str(row.subject), str(row.predicate), str(row.object)))
    return connections

In [110]:
def getPropertyTriples(graph, iri):
    return getPropertyTriplesWithSubject(graph, iri) + getPropertyTriplesWithObject(graph, iri)

In [111]:
classes = getAllClasses(physciGraph)
classNames = [getNodeName(physciGraph, cls) for cls in classes]
classDescriptions = [getNodeDescription(physciGraph, cls) for cls in classes]
classComments = [getNodeComment(physciGraph, cls) for cls in classes]
superclasses = [getSuperClasses(physciGraph, cls) for cls in classes]
propertyTriples = [getPropertyTriples(physciGraph, cls) for cls in classes]


In [112]:
print(classes)
print(classNames)
print(classDescriptions)
print(classComments)
print(superclasses)
print(propertyTriples)

['http://ecoinformatics.org/oboe/oboe.1.2/oboe-characteristics.owl#Amount', 'http://ecoinformatics.org/oboe/oboe.1.2/oboe-characteristics.owl#Energy', 'http://ecoinformatics.org/oboe/oboe.1.2/oboe-characteristics.owl#Force', 'http://ecoinformatics.org/oboe/oboe.1.2/oboe-characteristics.owl#Mass', 'http://ecoinformatics.org/oboe/oboe.1.2/oboe-characteristics.owl#MassDensity', 'http://ecoinformatics.org/oboe/oboe.1.2/oboe-characteristics.owl#Radiance', 'http://ecoinformatics.org/oboe/oboe.1.2/oboe-characteristics.owl#Speed', 'http://ecoinformatics.org/oboe/oboe.1.2/oboe-characteristics.owl#Volume', 'http://ecoinformatics.org/oboe/oboe.1.2/oboe-characteristics.owl#WaveNumber', 'http://ecoinformatics.org/oboe/oboe.1.2/oboe-core.owl#CategoricalStandard', 'http://ecoinformatics.org/oboe/oboe.1.2/oboe-core.owl#Characteristic', 'http://ecoinformatics.org/oboe/oboe.1.2/oboe-core.owl#CharacteristicQualifier', 'http://ecoinformatics.org/oboe/oboe.1.2/oboe-core.owl#Entity', 'http://ecoinformatics.

In [3]:
from json import dumps
def safeVisNetworkJSONToHTMLFile(jsonData, htmlFilePath):
    with open(htmlFilePath, "w") as htmlFile:
        htmlFile.write(
            f"""
            <!DOCTYPE html>
            <html lang="en-US">
            
            <head>
                <meta charset="UTF-8">
                <meta name="viewport" content="width=device-width, initial-scale=1">
                <title>Inline vis</title>
            </head>
            
            <body>
                <div id="mynetwork" style="width:100vw; height:100vh;"></div>
                <script src=" https://cdn.jsdelivr.net/npm/vis-data@7.1.9/peer/umd/vis-data.min.js "></script>
                <script src=" https://cdn.jsdelivr.net/npm/vis-network@9.1.9/peer/umd/vis-network.min.js "></script>
                <link href=" https://cdn.jsdelivr.net/npm/vis-network@9.1.9/styles/vis-network.min.css " rel="stylesheet">
                <script>
                    var jsonData = {dumps(jsonData)};
                    // create a network
                    var container = document.getElementById("mynetwork");
                    var options = {{}};
                    var network = new vis.Network(container, jsonData, options);
                </script>
            </body>
            
            </html>
            """
        )

In [22]:
def visualizeNamedIndividuals(graph, filename = "output.html"):
    namedIndividuals = getAllNamedIndividuals(graph)
    connections = getAllConnectionsBetweenNamedIndividuals(graph)
    nodes = []
    edges = []
    for namedIndividual in namedIndividuals:
        nodes.append({"id": namedIndividual, "label": getNodeName(graph, namedIndividual)})
    for connection in connections:
        edges.append({"from": connection[0], "to": connection[2], "label": getNodeName(graph, connection[1]), "arrows": "to"})
    visjsjson = {"nodes": nodes, "edges": edges}
    safeVisNetworkJSONToHTMLFile(visjsjson, filename)

In [ ]:
def visualizeOntology(graph, filename = "output.html"):
    classes = getAllClasses(graph)
    classNames = [getNodeName(graph, cls) for cls in classes]
    classDescriptions = [getNodeDescription(graph, cls) for cls in classes]
    classComments = [getNodeComment(graph, cls) for cls in classes]
    superclasses = [getSuperClasses(graph, cls) for cls in classes]
    propertyTriples = [getPropertyTriples(graph, cls) for cls in classes]
    nodes = []
    edges = []
    for i, cls in enumerate(classes):
        nodes.append({"id": cls, "label": classNames[i]})
        for superclass in superclasses[i]:
            edges.append({"from": cls, "to": superclass, "label": "subClassOf", "arrows": "to"})
        for triple in propertyTriples[i]:
            edges.append({"from": triple[0], "to": triple[2], "label": triple[1], "arrows": "to"})
    visjsjson = {"nodes": nodes, "edges": edges}
    safeVisNetworkJSONToHTMLFile(visjsjson, filename)

In [23]:
visualizeNamedIndividuals(physciGraph, "output.html")